# Análise de Logística e Previsão de Entregas (Olist)

## Visão Geral do Projeto

**Contexto de Negócio:**
A Olist conecta pequenas empresas a canais de vendas (marketplaces) em todo o Brasil. O maior desafio operacional nesse modelo não é apenas vender, mas garantir que o produto chegue ao cliente no prazo. Atrasos na entrega geram cancelamentos, churn e prejuízo à reputação da marca.

**Objetivo:**
Neste projeto, realizo uma análise completa da cadeia logística (Logistics Analytics) para identificar gargalos de performance e preparo os dados para o meu modelo preditivo de atrasos.

**Metodologia (Pipeline):**
Neste notebook, executo o processo de **ETL (Extract, Transform, Load)** e **EDA (Exploratory Data Analysis)** seguindo estas etapas:

1.  **Ingestão e Saneamento:** Unificação das tabelas relacionais e limpeza de dados inconsistentes (datas futuras, cadastros nulos).
2.  **Análise de Negócio com SQL:** Simulação de ambiente corporativo, persistindo os dados tratados em banco relacional para responder perguntas estratégicas via queries SQL.
3.  **Engenharia de Features:** Criação de variáveis estratégicas, como *volume cm3* e *Peso Cubado* (logística).
4.  **SLA Breakdown:** Decomposição do tempo total de entrega em etapas mensuráveis (Aprovação, Postagem/First Mile, Trânsito/Last Mile).
5.  **Arquitetura de Dados:** Bifurcação do dataset final em duas visões:
    * **Visão de BI (`df_dashb`):** Dados completos para análise de perdas e KPIs.
    * **Visão de Machine Learning (`df_model`):** Dados higienizados para treinamento de algoritmos.

## 1. Importação dos dados

In [1]:
import os
import json
import pandas as pd
import numpy as np
import zipfile

In [ ]:
with open('kaggle.json', 'r') as f:
    dados_json = json.load(f)
    os.environ['KAGGLE_USERNAME'] = dados_json['username']
    os.environ['KAGGLE_KEY'] = dados_json['key']
    
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

pasta_data = '../data'
os.makedirs(pasta_data, exist_ok=True)

api.dataset_download_files(
    'olistbr/brazilian-ecommerce', 
    path=pasta_data, 
    unzip=True, 
    force=True
)

print('Download e extração concluídos com sucesso!')

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
Download e extração concluídos com sucesso!


In [ ]:
arquivos = {
    'pedidos' : 'olist_orders_dataset.csv',
    'itens_pedido' : 'olist_order_items_dataset.csv',
    'produtos' : 'olist_products_dataset.csv',
    'clientes' : 'olist_customers_dataset.csv',
    'vendedores' : 'olist_sellers_dataset.csv'
}

dfs = {}

for nome, arquivo in arquivos.items():
    dfs[nome] = pd.read_csv(f'{pasta_data}/{arquivo}')

print('Tabelas carregadas na memória!')

Tabelas carregadas na memória!


## 2. Checagem de Qualidade dos Dados (EDA Inicial)

In [4]:
pk_map = {
    "pedidos": "order_id",
    "itens_pedido": ["order_id", "order_item_id"],
    "produtos": "product_id",
    "clientes": "customer_id",
    "vendedores": "seller_id"
}

def checagem_dados(df, nome, pk=None):
    print(f'\n--- Análise da Tabela: {nome} ---')
    print(f'Linhas: {df.shape[0]} | Colunas: {df.shape[1]}')

    if pk:
        if isinstance(pk, list):  # chave composta
            duplicados = df.duplicated(subset=pk).sum()
            print(f'Duplicados na chave {pk}: {duplicados}')
        else:  # chave simples
            duplicados = df[pk].duplicated().sum()
            print(f'Duplicados em {pk}: {duplicados}')
    else:
        print("Nenhuma chave primária definida para esta tabela.")

    total_nulos = df.isnull().sum().sum()
    print(f'Total de células vazias: {total_nulos}')

    if total_nulos > 0:
        print('\nColunas com mais nulos:')
        print(
            df.isnull()
              .sum()
              .sort_values(ascending=False)
        )

In [5]:
for nome, df in dfs.items():
    checagem_dados(df, nome, pk=pk_map.get(nome))


--- Análise da Tabela: pedidos ---
Linhas: 99441 | Colunas: 8
Duplicados em order_id: 0
Total de células vazias: 4908

Colunas com mais nulos:
order_delivered_customer_date    2965
order_delivered_carrier_date     1783
order_approved_at                 160
order_id                            0
order_purchase_timestamp            0
order_status                        0
customer_id                         0
order_estimated_delivery_date       0
dtype: int64

--- Análise da Tabela: itens_pedido ---
Linhas: 112650 | Colunas: 7
Duplicados na chave ['order_id', 'order_item_id']: 0
Total de células vazias: 0

--- Análise da Tabela: produtos ---
Linhas: 32951 | Colunas: 9
Duplicados em product_id: 0
Total de células vazias: 2448

Colunas com mais nulos:
product_category_name         610
product_description_lenght    610
product_name_lenght           610
product_photos_qty            610
product_weight_g                2
product_height_cm               2
product_length_cm               2
produ

### 2.1 Tratamento dos dados

Durante a varredura inicial das tabelas, identifiquei dados faltantes concentrados em dois datasets principais:

1.  **Tabela de Pedidos (`orders`):**
    * Apresenta lacunas críticas nas colunas temporais: `order_delivered_customer_date` (Data de Entrega), `order_delivered_carrier_date` (Data de Postagem) e `order_approved_at` (Data de Aprovação).

2.  **Tabela de Produtos (`products`):**
    * Contém itens com ausência de metadados de cadastro (`product_category_name`, `product_description_lenght`, `product_photos_qty`) e especificações logísticas (`product_weight_g`).

Abaixo, detalho minha estratégia de tratamento para cada caso.

#### Tratamento tabela de produtos.
Na minha checagem de qualidade, identifico dois grupos de problemas distintos:

**Problema A: Dados de Marketing Faltantes**
Encontrei *610* produtos sem categoria, descrição ou fotos. Embora isso seja um problema de cadastro (marketing), não gera impacto direto na operação logística. Contudo, não posso excluir esses dados, pois perderia o histórico de vendas.
* **Minha Solução:** Realizo a imputação dos valores. Preencho a categoria com "desconhecido" e atribuo zero (0) para comprimento de nome/descrição e quantidade de fotos.

**Problema B: Dados Logísticos Críticos Faltantes**
Identifiquei *2* produtos sem peso ou dimensões. Essas informações são essenciais para cálculo de frete e cubagem de carga.
* **Minha Solução:** Como são apenas 2 registros em milhares, opto pela exclusão, pois o impacto estatístico é nulo e garanto a integridade dos cálculos futuros.

In [6]:
dfs['produtos']['product_category_name'] = dfs['produtos']['product_category_name'].fillna('unknown')
print('Coluna category_name tratada!')

col_mkt = ['product_description_lenght', 'product_name_lenght', 'product_photos_qty']
dfs['produtos'][col_mkt] = dfs['produtos'][col_mkt].fillna(0)
print('Colunas numéricas de marketing tratadas!')

col_criticas = ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
dfs['produtos'] = dfs['produtos'].dropna(subset=col_criticas)
print('Colunas criticas removidas!')

print('Verificação final de nulos na tabela produtos:')
checagem_dados(dfs['produtos'], 'produtos', pk=pk_map.get('produtos'))

Coluna category_name tratada!
Colunas numéricas de marketing tratadas!
Colunas criticas removidas!
Verificação final de nulos na tabela produtos:

--- Análise da Tabela: produtos ---
Linhas: 32949 | Colunas: 9
Duplicados em product_id: 0
Total de células vazias: 0


#### Tratamento da tabela de Pedidos.
Identifiquei valores nulos em três colunas temporais críticas: `data_entrega` (2965 nulos), `data_entrega_transportadora` (1783 nulos) e `data_aprovacao` (160 nulos).

**Interpretação dos Nulos:**
Entendo que a ausência de data nem sempre é um erro de dados. Em muitos casos, reflete o status real do processo (ex: um pedido "cancelado" ou "em trânsito" naturalmente não tem data de entrega final).

**Minha Estratégia de Tratamento:**
1.  **Análise Cruzada:** Cruzo a ausência de datas com o `order_status` para distinguir o que é processo em andamento do que é erro de registro.
2.  **Conversão de Tipos:** Para evitar inconsistências nos cálculos de SLA, garanto que todas as 5 colunas de data sejam convertidas explicitamente para o formato `datetime`.

In [7]:
dfs['pedidos']['order_status'].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

In [8]:
print("Status dos pedidos sem data de entrega:")
display(dfs['pedidos'][dfs['pedidos']['order_delivered_customer_date'].isnull()]['order_status'].value_counts())

Status dos pedidos sem data de entrega:


order_status
shipped        1107
canceled        619
unavailable     609
invoiced        314
processing      301
delivered         8
created           5
approved          2
Name: count, dtype: int64

Para organizar melhor os dados, dividi a análise em três etapas principais:

1. Work in Progress (Em andamento): shipped, processing, invoiced
    -  Nesses casos, os valores nulos fazem sentido, já que o pedido ainda está em processo ou a caminho.
2. Loss (Perda): canceled, unavailable
    - Também é esperado que haja valores nulos, pois o pedido foi cancelado ou o produto estava indisponível.
3. Ruído: delivered (8 casos)
    - Aqui identifiquei um problema: o sistema marca como “Entregue”, mas a data está vazia. Isso provavelmente ocorreu por falta de registro da informação, gerando inconsistência e sujeira nos dados.

**Tratamento**: Decidi remover os casos classificados como ruído, já que são contraditórios e atrapalham a análise. Mantive todos os demais registros, pois eles trazem informações essenciais para o negócio, como taxa de cancelamento, pedidos em aberto e outros indicadores relevantes.

In [9]:
ruidos = (dfs['pedidos']['order_status'] == 'delivered') & (dfs['pedidos']['order_delivered_customer_date'].isnull())

dfs['pedidos'] = dfs['pedidos'][~ruidos]
print("Registros ruidosos removidos!")

cols_data = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in cols_data:
    dfs['pedidos'][col] = pd.to_datetime(dfs['pedidos'][col], errors='coerce')

print("\n Tabela Pedidos tratada e tipagem corrigida!")

Registros ruidosos removidos!

 Tabela Pedidos tratada e tipagem corrigida!


#### Análise de Consistência Cronológica (Sanity Check)

Antes de finalizar o tratamento da tabela de pedidos, realizo uma verificação de coerência temporal para garantir que a linha do tempo dos eventos faça sentido lógico. Essa etapa é crucial para evitar distorções no cálculo de *Lead Time*.

**Pontos de Verificação:**

1.  **Incoerência de Status (Envio sem Data):**
    * Investigo casos onde o status indica que o produto já saiu (`shipped` ou `delivered`), mas a `order_delivered_carrier_date` (data de postagem) não foi registrada. Isso aponta falha de integração sistêmica.

2.  **Paradoxo Temporal (Viagem no Tempo):**
    * Verifico se existem registros onde a **aprovação** ou a **postagem** ocorreram *antes* da data da compra (`order_purchase_timestamp`).
    * *Ex:* Pedido comprado dia 10/05, mas aprovado dia 09/05.

Essa checagem me permite detectar falhas de registro ou inconsistências sistêmicas que poderiam comprometer os indicadores logísticos e financeiros.

In [10]:
print('--- Análise de consistência cronológica ---')

cols_data_crono = ['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']
nulos_status = dfs['pedidos'].groupby('order_status')[cols_data_crono].apply(lambda x: x.isnull().sum())
print('\n Quantidade de nulos por status de pedido:')
display(nulos_status)

dt_nao_nulas = dfs['pedidos'].dropna(subset=['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date'])
erro_aprov = dt_nao_nulas[dt_nao_nulas['order_approved_at'] < dt_nao_nulas['order_purchase_timestamp']]
print(f'Casos de aprovação antes de compra: {len(erro_aprov)}')

erro_envio = dt_nao_nulas[dt_nao_nulas['order_delivered_carrier_date'] < dt_nao_nulas['order_approved_at']]
print(f'Casos de envio antes da aprovação: {len(erro_envio)}')

if len(erro_envio) > 0:
    print('\n Erros de envio antes da aprovação:')
    display(erro_envio[['order_status', 'order_approved_at', 'order_delivered_carrier_date']].head())
else:
    print('Nenhum erro de envio antes da aprovação encontrado.')

--- Análise de consistência cronológica ---

 Quantidade de nulos por status de pedido:


,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date
order_status,,,
approved,0,2,2
canceled,141,550,619
created,5,5,5
delivered,14,1,0
invoiced,0,314,314
processing,0,301,301
shipped,0,0,1107
unavailable,0,609,609


Casos de aprovação antes de compra: 0
Casos de envio antes da aprovação: 1359

 Erros de envio antes da aprovação:


,order_status,order_approved_at,order_delivered_carrier_date
15,delivered,2018-06-12 23:31:02,2018-06-11 14:54:00
64,delivered,2018-04-24 18:25:22,2018-04-23 19:19:14
199,delivered,2018-07-26 23:31:53,2018-07-24 12:57:00
210,delivered,2018-07-23 12:31:53,2018-07-23 12:24:00
415,delivered,2018-07-27 23:31:09,2018-07-24 14:03:00


Insights da análise:

1. - Nulos por status: identifiquei 14 pedidos marcados como entregue que estão com a data de aprovação vazia e 1 pedido entregue ao cliente sem data de entrega registrada na transportadora.
    - **Solução:** considerei essas situações como Data Ingestion Error. Como essa inconsistência me impede de calcular corretamente o tempo de aprovação, optei por remover esses registros para evitar ruído na análise.

2. Envio antes da aprovação: encontrei 1.359 pedidos que foram marcados como enviados antes mesmo de serem aprovados no sistema. Esse comportamento pode ocorrer em modelos de negócio que despacham rapidamente para ganhar SLA ou por erro de ingestão manual de dados.
    -  **Solução:** decidi manter esses registros. Na análise de SLA, vou considerar o tempo como 0, já que o processo não travou e o envio foi realizado.

In [11]:
delet_nulos_status = (
    (dfs['pedidos']['order_status'] == 'delivered') & 
    (dfs['pedidos']['order_approved_at'].isnull() | dfs['pedidos']['order_delivered_carrier_date'].isnull())
) 

print(f'Removendo {delet_nulos_status.sum()} pedidos com datas incompletas.')
dfs['pedidos'] = dfs['pedidos'][~delet_nulos_status]

Removendo 15 pedidos com datas incompletas.


#### Tratamento da tabela de Itens Pedidos.

- Tratamento da coluna de data
    - **Solução:** - garanto a tipagem correta das colunas de data, convertendo-as explicitamente para o formato datetime. Dessa forma, evito que sejam interpretadas como texto e asseguro que cálculos temporais e análises cronológicas sejam consistentes.

- Visualização estatística da sanidade financiera 
    - **Intenção:** verifico se os valores de preço e frete estão dentro da normalidade, garantindo que não existam valores negativos ou fora de escala. Essa checagem é essencial para confirmar a integridade dos dados financeiros e evitar distorções nas análises de custo e logística.

In [12]:
dfs['itens_pedido']['shipping_limit_date'] = pd.to_datetime(dfs['itens_pedido']['shipping_limit_date'], errors='coerce')
print("\n Tipagem corrigida!")


 Tipagem corrigida!


In [13]:
print('--- Análise descritiva dos preços e fretes dos itens do pedido ---\n')
print(dfs['itens_pedido'][['price', 'freight_value']].describe())

--- Análise descritiva dos preços e fretes dos itens do pedido ---

               price  freight_value
count  112650.000000  112650.000000
mean      120.653739      19.990320
std       183.633928      15.806405
min         0.850000       0.000000
25%        39.900000      13.080000
50%        74.990000      16.260000
75%       134.900000      21.150000
max      6735.000000     409.680000


Tudo está normal com os dados financeiros.

Não encontrei valores negativos e o valor mínimo de frete igual a zero é esperado, já que representa casos de frete grátis. Apenas identifiquei um valor máximo de frete bem acima do padrão, que pretendo analisar mais adiante. Por enquanto, vou manter todos os dados, pois não identifiquei nenhuma inconsistência ou erro.

## 3. Análise de Negócio com SQL (Simulação de Ambiente Corporativo)

Embora o *Pandas* seja uma ferramenta poderosa para manipulação de dados, a grande maioria das empresas armazena seus dados históricos em *Bancos de Dados Relacionais (SQL)* ou em Data Warehouses (BigQuery, Redshift, etc.). Para simular um ambiente de produção realista e demonstrar minha competência na extração de dados, defini a seguinte estratégia:

1. 	**Persistência:** após realizar todos os tratamentos necessários para garantir uma base de dados limpa e coerente, carrego os DataFrames tratados em um banco de dados local temporário utilizando a biblioteca nativa sqlite3, que me permite transformar o Python em um banco de dados SQL real.
2. 	**Consultas (Querying):** utilizo exclusivamente Linguagem SQL para responder às perguntas estratégicas de negócio.

Nesse processo, primeiro crio um banco de dados em memória e uso esse ambiente para executar os joins e consolidar tudo em uma única tabela analítica. Em seguida, aplico o comando *PRAGMA table_info*, que me permite inspecionar o esquema do banco de dados. Essa tabela de metadados descreve cada coluna existente e é fundamental na fase de verificação do ETL, pois assegura a qualidade da etapa de carga.


In [14]:
import sqlite3

In [15]:
conn = sqlite3.connect(':memory:')

for nome, df in dfs.items():
    df.to_sql(nome, conn, index=False, if_exists='replace')
   
print('Tabelas carregadas no banco de dados SQL na memória!') 

tabelas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

for tabela in tabelas['name']:
    print(f'\n --- estrutura da tabela: {tabela} ---')
    display(pd.read_sql(f"PRAGMA table_info({tabela});", conn))

Tabelas carregadas no banco de dados SQL na memória!

 --- estrutura da tabela: pedidos ---


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,TEXT,0,None,0
1,1,customer_id,TEXT,0,None,0
2,2,order_status,TEXT,0,None,0
3,3,order_purchase_timestamp,TIMESTAMP,0,None,0
4,4,order_approved_at,TIMESTAMP,0,None,0
5,5,order_delivered_carrier_date,TIMESTAMP,0,None,0
6,6,order_delivered_customer_date,TIMESTAMP,0,None,0
7,7,order_estimated_delivery_date,TIMESTAMP,0,None,0



 --- estrutura da tabela: itens_pedido ---


,cid,name,type,notnull,dflt_value,pk
0,0,order_id,TEXT,0,None,0
1,1,order_item_id,INTEGER,0,None,0
2,2,product_id,TEXT,0,None,0
3,3,seller_id,TEXT,0,None,0
4,4,shipping_limit_date,TIMESTAMP,0,None,0
5,5,price,REAL,0,None,0
6,6,freight_value,REAL,0,None,0



 --- estrutura da tabela: produtos ---


,cid,name,type,notnull,dflt_value,pk
0,0,product_id,TEXT,0,None,0
1,1,product_category_name,TEXT,0,None,0
2,2,product_name_lenght,REAL,0,None,0
3,3,product_description_lenght,REAL,0,None,0
4,4,product_photos_qty,REAL,0,None,0
5,5,product_weight_g,REAL,0,None,0
6,6,product_length_cm,REAL,0,None,0
7,7,product_height_cm,REAL,0,None,0
8,8,product_width_cm,REAL,0,None,0



 --- estrutura da tabela: clientes ---


,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,0
1,1,customer_unique_id,TEXT,0,None,0
2,2,customer_zip_code_prefix,INTEGER,0,None,0
3,3,customer_city,TEXT,0,None,0
4,4,customer_state,TEXT,0,None,0



 --- estrutura da tabela: vendedores ---


,cid,name,type,notnull,dflt_value,pk
0,0,seller_id,TEXT,0,None,0
1,1,seller_zip_code_prefix,INTEGER,0,None,0
2,2,seller_city,TEXT,0,None,0
3,3,seller_state,TEXT,0,None,0


Para realizar o merge, eu seleciono somente as tabelas e colunas que são realmente relevantes para minha análise. Em seguida, renomeio os nomes das colunas para português, tornando-os mais claros, amigáveis e concisos. Essa padronização facilita tanto a leitura quanto a interpretação dos resultados, além de deixar o processo analítico mais acessível e consistente.

In [16]:
query_tabela_logistica = """
SELECT
    o.order_id AS pedido_id,
    o.order_status AS status_pedido,
    o.order_purchase_timestamp AS data_compra,
    o.order_approved_at AS data_aprovacao,
    o.order_delivered_carrier_date AS data_postagem,
    o.order_delivered_customer_date AS data_entrega,
    o.order_estimated_delivery_date AS data_estimada,

    i.price AS preco_produto,
    i.freight_value AS valor_frete,
    i.shipping_limit_date AS data_limite_postagem,

    p.product_weight_g AS peso_produto_g,
    p.product_length_cm AS comprimento_produto_cm,
    p.product_height_cm AS altura_produto_cm,
    p.product_width_cm AS largura_produto_cm,
    p.product_category_name AS categoria_produto,

    c.customer_zip_code_prefix AS cep_cliente,
    c.customer_city AS cidade_cliente,
    c.customer_state AS uf_cliente,

    v.seller_zip_code_prefix AS cep_vendedor,
    v.seller_city AS cidade_vendedor,
    v.seller_state AS uf_vendedor,

    -- Cria Flag de Pickup
    CASE
        WHEN c.customer_city = v.seller_city AND c.customer_state = v.seller_state THEN 1
        ELSE 0
    END AS flag_pickup

FROM pedidos o
LEFT JOIN itens_pedido i ON o.order_id = i.order_id
LEFT JOIN produtos p ON i.product_id = p.product_id
LEFT JOIN clientes c ON o.customer_id = c.customer_id
LEFT JOIN vendedores v ON i.seller_id = v.seller_id;
"""

df_analise = pd.read_sql_query(query_tabela_logistica, conn)

cols_data_final = ['data_compra', 'data_aprovacao', 'data_postagem', 'data_entrega', 'data_estimada', 'data_limite_postagem']
for col in cols_data_final:
    df_analise[col] = pd.to_datetime(df_analise[col], errors='coerce')

df_analise.to_sql('analise_logistica', conn, if_exists='replace', index=False)

print('Sucesso! Tabela "analise_logistica" criada/atualizada via Pandas.')
display(df_analise.head())

Sucesso! Tabela "analise_logistica" criada/atualizada via Pandas.


,pedido_id,status_pedido,data_compra,data_aprovacao,data_postagem,data_entrega,data_estimada,preco_produto,valor_frete,data_limite_postagem,...,altura_produto_cm,largura_produto_cm,categoria_produto,cep_cliente,cidade_cliente,uf_cliente,cep_vendedor,cidade_vendedor,uf_vendedor,flag_pickup
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,29.99,8.72,2017-10-06 11:07:15,...,8.0,13.0,utilidades_domesticas,3149,sao paulo,SP,9350.0,maua,SP,0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,118.70,22.76,2018-07-30 03:24:27,...,13.0,19.0,perfumaria,47813,barreiras,BA,31570.0,belo horizonte,SP,0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,159.90,19.22,2018-08-13 08:55:23,...,19.0,21.0,automotivo,75265,vianopolis,GO,14840.0,guariba,SP,0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,45.00,27.20,2017-11-23 19:45:59,...,10.0,20.0,pet_shop,59296,sao goncalo do amarante,RN,31842.0,belo horizonte,MG,0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,19.90,8.72,2018-02-19 20:31:37,...,15.0,15.0,papelaria,9195,santo andre,SP,8752.0,mogi das cruzes,SP,0


Tabela de análise logistica principal criada, vamos validar com uma query de teste.

In [17]:
query_teste = "SELECT * FROM analise_logistica LIMIT 5"
display(pd.read_sql_query(query_teste, conn))

,pedido_id,status_pedido,data_compra,data_aprovacao,data_postagem,data_entrega,data_estimada,preco_produto,valor_frete,data_limite_postagem,...,altura_produto_cm,largura_produto_cm,categoria_produto,cep_cliente,cidade_cliente,uf_cliente,cep_vendedor,cidade_vendedor,uf_vendedor,flag_pickup
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,29.99,8.72,2017-10-06 11:07:15,...,8.0,13.0,utilidades_domesticas,3149,sao paulo,SP,9350.0,maua,SP,0
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,118.70,22.76,2018-07-30 03:24:27,...,13.0,19.0,perfumaria,47813,barreiras,BA,31570.0,belo horizonte,SP,0
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,159.90,19.22,2018-08-13 08:55:23,...,19.0,21.0,automotivo,75265,vianopolis,GO,14840.0,guariba,SP,0
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,45.00,27.20,2017-11-23 19:45:59,...,10.0,20.0,pet_shop,59296,sao goncalo do amarante,RN,31842.0,belo horizonte,MG,0
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,19.90,8.72,2018-02-19 20:31:37,...,15.0,15.0,papelaria,9195,santo andre,SP,8752.0,mogi das cruzes,SP,0


In [18]:
query_pickup = """
SELECT 
    flag_pickup,
    COUNT(*) AS qtd_pedidos,
    AVG(valor_frete) AS media_frete
FROM analise_logistica
WHERE status_pedido = 'delivered'
GROUP BY flag_pickup
"""

display(pd.read_sql_query(query_pickup, conn))

,flag_pickup,qtd_pedidos,media_frete
0,0,104465,20.444584
1,1,5708,10.880044


### 3.1 Consultas em SQL

Com a base analítica consolidada no banco de dados SQLite, passo a realizar consultas em SQL para responder às principais perguntas de negócio. Essa etapa simula o ambiente corporativo, onde a extração de informações estratégicas é feita diretamente via linguagem SQL.

#### 3.1.1 Os KPIs Gerais. 
    - Um resumo da saúde da operação.

Qual é a taxa de atraso, tempo médio de entrega e ticket médio de frete?

In [19]:
query_kpis = """
SELECT 
    COUNT(pedido_id) AS total_pedidos_entregues,
    
    -- Taxa de atraso (Se entrega > estiimativa = 1, senão 0)
    ROUND(AVG(CASE
        WHEN data_entrega > data_estimada THEN 1
        ELSE 0
    END) * 100, 2) AS perc_atraso,
    
    -- tempo médio de entrega
    ROUND(AVG(JULIANDAY(data_entrega) - JULIANDAY(data_compra)), 2) AS tempo_medio_entrega,
    
    -- Calculo custo médio de frete
    ROUND(AVG(valor_frete), 2) AS custo_medio_frete
    
FROM analise_logistica     
"""

print('\n --- Visão geral de KPIs ---\n')
display(pd.read_sql_query(query_kpis, conn))


 --- Visão geral de KPIs ---



,total_pedidos_entregues,perc_atraso,tempo_medio_entrega,custo_medio_frete
0,113401,7.68,12.47,19.99


Na análise da saúde geral do negócio, os indicadores logísticos revelam um *tempo médio de entrega de 12,5 dias*, um *custo médio de frete de R$ 19,99* e um *percentual de atraso de 7,6%*.

O ponto mais crítico é o índice de atraso: quase 8% é elevado em operações de transporte e impacta diretamente a experiência do cliente. Entregas fora do prazo geram frustração, aumentam a abertura de chamados no suporte (e consequentemente os custos de atendimento) e prejudicam a reputação da empresa.

Uma possível causa está na forma como o algoritmo de previsão de entrega calcula a data estimada. Atualmente, ele parece estar otimista demais, prometendo prazos inferiores ao tempo médio real de transporte.

Para mitigar esse problema, é recomendável recalibrar a previsão de entrega, ajustando-a para valores mais realistas. Dessa forma, a empresa pode criar uma margem de segurança: prometer um prazo maior e entregar antes do previsto, gerando uma percepção positiva e fortalecendo a confiança do cliente.

#### 3.1.2 SLA Breadown 
    - Onde está o gargalo?

Onde estamos perdendo tempo? No armazém (vendedor) ou no transporte?

In [20]:
query_sla = """
SELECT
    -- 1. Tempo de aprovação (Financeiro)
    ROUND(AVG(JULIANDAY(data_aprovacao) - JULIANDAY(data_compra)), 2) AS tempo_medio_aprovacao,
    
    -- 2. Tempo de processamento (Eficiência operacional)
    ROUND(AVG(JULIANDAY(data_postagem) - JULIANDAY(data_aprovacao)), 2) AS tempo_medio_processamento,
    
    -- 3. Tempo de transporte (Eficiência logística)
    ROUND(AVG(JULIANDAY(data_entrega) - JULIANDAY(data_postagem)), 2) AS tempo_medio_transporte,
    
    -- Tempo total de entrega
    ROUND(AVG(JULIANDAY(data_entrega) - JULIANDAY(data_compra)), 2) AS tempo_total_entrega
    
FROM analise_logistica
WHERE status_pedido = 'delivered'
"""

print('--- Análise dos SLAs ---\n')
display(pd.read_sql_query(query_sla, conn))

--- Análise dos SLAs ---



,tempo_medio_aprovacao,tempo_medio_processamento,tempo_medio_transporte,tempo_total_entrega
0,0.44,2.85,9.19,12.47


A avaliação dos prazos evidencia um **gargalo no tempo de processamento**: são quase *3 dias gastos apenas para a postagem do produto*, o que representa *23% do prazo total de entrega*. Essa demora reflete uma *ineficiência operacional do vendedor*.

A principal oportunidade de otimização está na *implementação de um serviço de Fulfillment*, que reduziria o tempo de processamento para cerca de 0,5 dia. Com isso, o prazo total cairia de 12 para 10 dias, sem necessidade de contratar transportadoras mais rápidas ou mais caras.

Outro ponto crítico é o *tempo total de entrega*. Em comparação com o mercado, um prazo médio de 12 dias é considerado elevado e funciona como barreira comercial, levando muitos clientes a abandonar o carrinho.

Por outro lado, o *frete barato (média de R$ 19,99)* torna esse prazo mais aceitável para parte dos consumidores. Ainda assim, há espaço para estratégia: oferecer uma opção de *entrega expressa*, com custo ligeiramente maior, atenderia clientes que priorizam rapidez e reduziria a taxa de abandono.

#### 3.1.3 Oportunidade de "Pickup" (Last Mile)
    - Quanto gastamos de frete enviando produtos dentro da mesma cidade que poderiam ser retirados?

In [21]:
query_pickup = """
SELECT    
    CASE 
        WHEN flag_pickup = 1 THEN 'Rota Local'
        ELSE 'Rota Longa'
    END AS tipo_rota,
    
    COUNT(DISTINCT pedido_id) AS qtd_pedidos,
    
    -- Soma do dinheiro gasto (Oportunidade de Economia)
    'R$ ' || ROUND(SUM(valor_frete), 2) AS total_gasto_frete,
    'R$ ' || ROUND(AVG(valor_frete), 2) AS frete_medio,
    
    -- Tempo médio para entrega
    ROUND(AVG(JULIANDAY(data_entrega) - JULIANDAY(data_postagem)), 1) AS tempo_medio_entrega

FROM analise_logistica 
GROUP BY flag_pickup
ORDER BY total_gasto_frete ASC
"""

print("--- KPI 3: Análise de Last Mile (Pickup) ---")
display(pd.read_sql_query(query_pickup, conn))

--- KPI 3: Análise de Last Mile (Pickup) ---


,tipo_rota,qtd_pedidos,total_gasto_frete,frete_medio,tempo_medio_entrega
0,Rota Longa,94396,R$ 2187884.31,R$ 20.49,9.5
1,Rota Local,5090,R$ 63596.34,R$ 10.87,2.8


A análise de Last Mile (oportunidade de Pickup) revela que *5% dos pedidos (5.090) correspondem a rotas locais*, responsáveis por um *custo logístico total de R$ 63.596,34*. O *frete médio de R$ 10,87* nessas rotas é considerado ineficiente, sobretudo quando comparado à complexidade das rotas longas, cujo frete médio é de R$ 20,49. Além disso, o *tempo médio de entrega de quase 3 dias* para regiões próximas evidencia uma lacuna operacional relevante.

Esse cenário abre espaço para uma *oportunidade clara de melhoria financeira e de nível de serviço*:
- *Pickup (retirada em loja)*: elimina o custo logístico e reduz o prazo de entrega a zero, fortalecendo a percepção de conveniência.
- *Frete local otimizado (same-day delivery)*: renegociar modalidades de entrega urbana permitiria reduzir o tempo de entrega e melhorar a eficiência do custo médio.

Com essas iniciativas, a empresa pode elevar a satisfação do cliente, reduzir custos e criar diferenciais competitivos na última etapa da jornada de compra.


#### 3.1.4 Ranking de Atrasos por Estado
    - Uma visão geográfica. Quais estados têm a pior performance de entrega?

In [22]:
query_atrasos = """
SELECT    
    uf_cliente,
    
    -- 1. Frequência de atraso
        COUNT(DISTINCT pedido_id) AS total_pedidos,
        SUM(CASE WHEN data_entrega > data_estimada THEN 1 ELSE 0 END) AS pedidos_atrasados,
        ROUND(100.0 * SUM(CASE WHEN data_entrega > data_estimada THEN 1 ELSE 0 END) / COUNT(DISTINCT pedido_id), 2) AS perc_atraso,
        
        -- 2. Média de dias de atraso
        ROUND(AVG(CASE 
            WHEN data_entrega > data_estimada THEN JULIANDAY(data_entrega) - JULIANDAY(data_estimada) 
            ELSE NULL 
        END), 1) AS media_dias_atraso
        
FROM analise_logistica
WHERE status_pedido = 'delivered'
GROUP BY uf_cliente
ORDER BY perc_atraso DESC
LIMIT 10
"""

print('--- Top 10 Estados com Maior Percentual de Atrasos ---')
display(pd.read_sql_query(query_atrasos, conn))

--- Top 10 Estados com Maior Percentual de Atrasos ---


,uf_cliente,total_pedidos,pedidos_atrasados,perc_atraso,media_dias_atraso
0,AL,397,103,25.94,9.2
1,MA,716,163,22.77,9.4
2,SE,335,61,18.21,15.2
3,CE,1278,218,17.06,13.8
4,PI,476,81,17.02,11.9
5,BA,3256,504,15.48,10.9
6,RJ,12348,1835,14.86,12.6
7,TO,274,38,13.87,5.8
8,PA,946,131,13.85,12.1
9,ES,1995,272,13.63,10.2


A análise logística evidencia que os maiores problemas de atraso estão concentrados nas regiões *Nordeste e Norte*, com exceções pontuais no Sudeste. Estados como *Alagoas (25,9%) e Maranhão (22,7%)* apresentam índices alarmantes, onde praticamente um em cada quatro pedidos não chega no prazo. Esse padrão sugere que o problema não se limita à última milha, mas pode estar relacionado a fatores estruturais, como a distância dos Centros de Distribuição ou barreiras fiscais que retardam a transferência de mercadorias.

Além da frequência, é essencial observar a *severidade dos atrasos*. Em *Sergipe*, os clientes enfrentam em média *15,2 dias adicionais de espera*, enquanto no *Ceará* a média é de *13,8 dias*. Isso significa que, embora em Alagoas o atraso seja mais provável, em Sergipe e Ceará o impacto é mais doloroso, gerando frustração intensa, queda na satisfação (NPS baixo) e aumento de custos com suporte, já que clientes permanecem semanas aguardando solução.

No *Rio de Janeiro*, o desafio é diferente: o percentual de atraso é menor (14,86%), mas o *volume de pedidos é massivo (12.348)*. Isso representa *1.835 clientes insatisfeitos*, número absoluto muito superior ao de outros estados. Em termos práticos, resolver apenas 1% de eficiência no RJ gera impacto operacional e financeiro maior do que resolver 10% em estados de menor escala, como Tocantins.

**Em resumo**, o quadro mostra três frentes de ação: revisão estrutural da malha logística para o Nordeste/Norte, ajuste de promessas de prazo e comunicação em estados com atrasos severos como SE e CE, e priorização de melhorias no RJ, onde o volume transforma pequenas ineficiências em grandes riscos para a reputação e custos da operação.


### 4. FEATURE ENGINEERING

Nesta etapa, traduzo os dados brutos em inteligência logística. Meu objetivo aqui não é apenas limpar colunas, mas criar novas features que expliquem a realidade operacional e permitam ao modelo matemático entender onde e por que os atrasos ocorrem.

In [23]:
df_analise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113401 entries, 0 to 113400
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   pedido_id               113401 non-null  object        
 1   status_pedido           113401 non-null  object        
 2   data_compra             113401 non-null  datetime64[ns]
 3   data_aprovacao          113255 non-null  datetime64[ns]
 4   data_postagem           111435 non-null  datetime64[ns]
 5   data_entrega            110180 non-null  datetime64[ns]
 6   data_estimada           113401 non-null  datetime64[ns]
 7   preco_produto           112626 non-null  float64       
 8   valor_frete             112626 non-null  float64       
 9   data_limite_postagem    112626 non-null  datetime64[ns]
 10  peso_produto_g          112608 non-null  float64       
 11  comprimento_produto_cm  112608 non-null  float64       
 12  altura_produto_cm       112608

#### 4.1 Tratamento de Dados Ausentes (Imputação Inteligente)

Identifiquei uma quantidade significativa de produtos com dimensões ou categorias nulas. Diante disso, tomei a decisão estratégica de não excluir esses registros, pois isso mascararia problemas reais de cadastro e reduziria o histórico de vendas.

Minha abordagem de imputação foi:

* **Categorias:** Preenchi os valores nulos como `'desconhecido'`. Isso permite que o modelo isole e aprenda o comportamento logístico específico desses itens mal cadastrados.
* **Dimensões:** Optei pela imputação via *Mediana da Categoria*. Assim, um móvel sem dimensões cadastradas recebe o tamanho "típico" de um móvel, evitando distorções no cálculo de frete que ocorreriam se eu usasse uma média global ou preenchesse com zeros.

In [24]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

df_analise['categoria_produto'] = df_analise['categoria_produto'].fillna('desconhecido')

cols_dimensao = ['comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm', 'peso_produto_g']

for col in cols_dimensao:
    df_analise[col] = df_analise.groupby('categoria_produto')[col].transform(lambda x: x.fillna(x.median()))
    df_analise[col] = df_analise[col].fillna(df_analise[col].median())
    
warnings.resetwarnings()

print("Tratamento de Nulos Concluído.")
print(f"Verificação de nulos restantes:\n", df_analise.isnull().sum().loc[lambda x: x > 0])

Tratamento de Nulos Concluído.
Verificação de nulos restantes:
 data_aprovacao           146
data_postagem           1966
data_entrega            3221
preco_produto            775
valor_frete              775
data_limite_postagem     775
cep_vendedor             775
cidade_vendedor          775
uf_vendedor              775
dtype: int64


#### 4.2 Engenharia Logística: Cubagem e Montagem de Carga

Compreendo que, no setor logístico, o peso físico (gramas) raramente é o único gargalo. O volume ocupado (cm³) é frequentemente o fator limitante para o dimensionamento da frota (decidir entre enviar carreta, caminhão ou van).

Para criar um modelo capaz de auxiliar na **otimização de carga**, fui além do peso em gramas. Criei duas novas variáveis cruciais:
1.  **`vol_cm3`**: O volume total do pedido.
2.  **`peso_cubado`**: Apliquei o fator de cubagem padrão rodoviário (300). Essa variável permite ao modelo entender o "peso taxável" e a ocupação real do veículo, refinando a previsão de complexidade da entrega.

In [25]:
df_analise['vol_cm3'] = df_analise['comprimento_produto_cm'] * df_analise['altura_produto_cm'] * df_analise['largura_produto_cm']
print("Cálculo do volume em m³ adicionado com sucesso!")

df_analise['peso_cubado_kg'] = (df_analise['vol_cm3'] / 1_000_000) * 300
print("Cálculo do peso cubado em kg adicionado com sucesso!")

Cálculo do volume em m³ adicionado com sucesso!
Cálculo do peso cubado em kg adicionado com sucesso!


#### 4.3 SLA Breakdown: Mapeamento de Gargalos

Para ir além de simplesmente prever *se vai atrasar*, precisei entender onde o tempo é gasto. Decompus o ciclo de vida do pedido em métricas de dias corridos, permitindo identificar a raiz do problema:
- **`tempo_aprovacao`** (Gargalo financeiro): Tempo entre a compra e a confirmação do pagamento.
- **`tempo_postagem`** (First Mile): A eficiência do vendedor em despachar o produto após a aprovação.
- **`tempo_transporte`** (Last Mile): A eficiência da transportadora em levar o pacote até o cliente.
- **`dias_atraso`** (Target): A diferença real entre a data de entrega e a data prometida (onde valores negativos indicam sucesso/antecipação).


In [26]:
df_analise['tempo_aprovacao'] = ((df_analise['data_aprovacao'] - df_analise['data_compra']).dt.total_seconds() / 86400).round(2)

df_analise['tempo_postagem'] = ((df_analise['data_postagem'] - df_analise['data_aprovacao']).dt.total_seconds() / 86400).round(2)
df_analise['tempo_postagem'] = df_analise['tempo_postagem'].clip(lower=0)

df_analise['tempo_transporte'] = ((df_analise['data_entrega'] - df_analise['data_postagem']).dt.total_seconds() / 86400).round(2)

df_analise['prazo_prometido'] = (df_analise['data_estimada'] - df_analise['data_compra']).dt.days

df_analise['dias_atraso'] = (df_analise['data_entrega'] - df_analise['data_estimada']).dt.days

df_analise['flag_atraso'] = np.where(df_analise['dias_atraso'] > 0, 1, 0)

print("Métricas de SLA calculadas com sucesso!")

Métricas de SLA calculadas com sucesso!


#### 4.4 **Separação de Estratégias: BI vs. Modelagem**

Ao analisar a consistência dos dados, percebi que tenho dois objetivos conflitantes. Para resolvê-los, adotei uma estratégia de bifurcação do dataset:

1.  **Visão de Negócio** (`df_dash`): Mantive intencionalmente os dados "sujos" (cancelamentos, falhas de vendedor e indisponibilidade). Preciso desses dados no Dashboard para calcular indicadores de perda (Loss Rate) e monitorar a saúde do cadastro.
2.  **Visão do Modelo** (`df_model`): Criei um subconjunto higienizado, removendo pedidos sem origem (vendedor) ou sem rota definida. Isso garante que o algoritmo de Machine Learning treine apenas com dados consistentes, evitando ruídos que degradariam a performance preditiva.

In [ ]:
df_dash = df_analise.copy()

df_dash['status_cadastro'] = np.where(
    df_dash['cep_vendedor'].isnull(), 
    'Incompleto (Sem Vendedor)', 
    'Completo'
)

df_model = df_analise.dropna(subset=['cep_vendedor', 'data_entrega']).copy()

arquivos_para_salvar = {
    'data_dashboard_processed.csv': df_dash,
    'data_model_processed.csv': df_model
}

for nome_arquivo, df in arquivos_para_salvar.items():
    caminho_final = os.path.join(pasta_data, nome_arquivo)
    df.to_csv(caminho_final, index=False)
    print(f"Salvo: {caminho_final}")

print(f"Processo concluído! {len(arquivos_para_salvar)} arquivos salvos em '{pasta_data}'.")

Salvo: ./data\data_dashboard_processed.csv
Salvo: ./data\data_model_processed.csv
Processo concluído! 2 arquivos salvos em './data'.


## 5. Conclusão do Módulo 1: Preparação e Engenharia de Dados

Neste notebook, finalizei a etapa crucial de **ETL (Extract, Transform, Load)** e **EDA (Exploratory Data Analysis)**. Mais do que apenas limpar dados, construí a inteligência de negócio que servirá de alicerce para as próximas fases do projeto.

### Entregáveis deste Notebook

Gerei dois artefatos distintos, cada um atendendo a um propósito estratégico:

1.  **`data_dashboard_processed.csv` (Visão de Negócio):**
    * Contém **todo o histórico operacional**, incluindo pedidos cancelados, falhas de cadastro e problemas de estoque.
    * **Objetivo:** Alimentar o Dashboard de Business Intelligence para cálculo de KPIs de *Loss Rate* (Perdas), *Churn* de pedidos e gargalos de faturamento.

2.  **`data_model_processed.csv` (Visão do Modelo):**
    * Base **higienizada e enriquecida**. Removi ruídos (pedidos sem vendedor/rota) e adicionei variáveis preditivas fortes como `peso cubado`, `tempo_aprovacao` e `flag de mesma cidade`.
    * **Objetivo:** Treinar o modelo preditivo para estimar prazos de entrega com alta precisão.

### Próximos Passos

Com a engenharia de dados concluída, o projeto segue para a fase de **Machine Learning**. No próximo notebook (`Modelagem_Logistica.ipynb`), irei carregar a base limpa para realizar:
* **Pré-processamento Matemático:** Encoding de variáveis categóricas e normalização de escalas.
* **Treinamento de Algoritmos:** Teste de modelos (Random Forest, XGBoost) para prever atrasos.
* **Avaliação de Performance:** Medição do erro médio (RMSE/MAE) das previsões.